In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
BATCH_SIZE = 128
TEST_SIZE = 256

def init_weights(shape):
    W = tf.Variable(tf.random_normal(shape = shape, stddev=0.01))
    return W

def get_model(X, w1, w2, w3, w4, w0, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1, 
                                  strides = [1, 1, 1, 1], 
                                  padding = 'SAME'))
    l1 = tf.nn.max_pool(l1a, ksize = [1, 2, 2, 1], 
                        strides = [1, 2, 2, 1],
                       padding = "SAME")
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, 
                                  strides = [1, 1, 1, 1],
                                 padding = 'SAME'))
    l2 = tf.nn.max_pool(l2a, 
                        ksize = [1, 2, 2, 1], 
                        strides = [1, 2, 2, 1],
                       padding = "SAME")
    l2 = tf.nn.dropout(l2, p_keep_hidden)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, 
                                 strides = [1, 1, 1, 1],
                                padding = "SAME")) # l3a shape=(?, 7, 7, 128)
    l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2, 1],
                       strides = [1, 2, 2, 1],
                       padding = "SAME")# l3 shape=(?, 4, 4, 128)
    l3 = tf.nn.dropout(l3, keep_prob=p_keep_hidden)
    
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]]) #reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    pyx = tf.matmul(l4, w0)
    return pyx
    
    
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = get_model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [ ]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), BATCH_SIZE),
                             range(BATCH_SIZE, len(trX)+1, BATCH_SIZE))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:TEST_SIZE]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))